[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/seoul-bike-station-location.ipynb)

# 따릉이 자전거 대여소 위치
* 크롤링한 전체 대여소 위치를 분석해 봅니다.

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import seaborn as sns

# 정규표현식 사용을 위해
import re

In [6]:
# 아나콘다에서 folium 을 사용하기 위해서는 별도의 설치가 필요
# https://anaconda.org/conda-forge/folium
# conda install -c conda-forge folium 
# 지도 표현을 위해
import folium

## 한글폰트 설정

In [7]:
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
# Window 의 한글 폰트 설정
# plt.rc('font',family='Malgun Gothic')
# Mac 의 한글 폰트 설정
plt.rc('font', family='AppleGothic') 
plt.rc('axes', unicode_minus=False)

set_matplotlib_formats('retina')

%matplotlib inline

## Colab 에서 실행을 위한 코드
* colab : https://colab.research.google.com/github/corazzon/seoul-bike-analysis/blob/master/seoul-bike-station-location.ipynb
* 아래의 코드는 google colaboratory 에서 실행을 위한 코드로 로컬 아나콘다에서는 주석처리한다.
* google colaboratory 에서는 주석을 풀고 폰트 설정과 csv 파일을 불러온다.

In [8]:
# # 나눔고딕 설치
# !apt -qq -y install fonts-nanum > /dev/null

# import matplotlib.font_manager as fm

# fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# font = fm.FontProperties(fname=fontpath, size=9)
# fm._rebuild()

# # 그래프에 retina display 적용
# %config InlineBackend.figure_format = 'retina'

# # Colab 의 한글 폰트 설정
# plt.rc('font', family='NanumBarunGothic') 

In [ ]:
# # 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증을 한다.
# !pip install -U -q PyDrive
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # PyDrive client 인증
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

In [ ]:
# # 공유 가능한 링크로 파일 가져오기
# url ='https://drive.google.com/open?id=1PbU3obWSNc7ADD2sAF2Anhb9HYH8JsDL'
# id = url.split('=')[1]
# print(id)
# downloaded = drive.CreateFile({'id':id}) 
# # data폴더에 데이터를 따로 모아 관리한다.
# %mkdir data
# downloaded.GetContentFile('data/seoul_bike_station.csv')  

## 파일로드

In [9]:
df = pd.read_csv('data/seoul_bike_station.csv')
df.shape

(1531, 6)

In [10]:
df.dtypes

대여소번호    float64
대여소       object
상태        object
주소        object
위도       float64
경도       float64
dtype: object

In [ ]:
# head 로 미리보기를 합니다. 기본값은 5입니다.
df.head()

In [ ]:
# tail로 미리보기를 합니다. 기본값은 5입니다.
df.tail()

In [ ]:
# info를 사용하면 데이터의 타입, 크기, 메모리 사용량 등을 볼 수 있습니다.
df.info()

In [ ]:
# 결측치를 봅니다.
df.isnull().sum()

In [ ]:
# 데이터프레임의 행과 열의 갯수를 출력합니다.
df.shape

In [ ]:
# value_counts 를 하게 되면 그룹화 된 데이터의 갯수를 세어줍니다.
df['상태'].value_counts()

In [ ]:
# 연산을 통해 위도가 37보다 크고, 경도가 125 보다 큰 값을 가져옵니다.
# 위도와 경도가 잘못 들어가 있는 값을 제거하고 가져오기 위함입니다.
geo_df = df.loc[(df['위도'] > 37) & (df['경도'] > 125)]
geo_df.shape

In [ ]:
# Pandas의 scatter plot으로 시각화를 합니다.
geo_df.plot.scatter(x='경도', y='위도', figsize=(10, 7))

In [ ]:
# 위와 같은 그래프이지만 seaborn 으로 시각화를 합니다.
plt.figure(figsize=(10, 7))
sns.scatterplot(data=geo_df, x="경도", y="위도")

## 구별 데이터 만들기 
* 공백으로 문자를 분리해서 1번째 인덱스에 있는 문자를 가져오도록 구를 생성하였다.
* 그런데 두 번째 문자가 구가 아닌 경우가 있다. 
* 이런 문자들을 개선하기 위해 공백 두 개는 하나로 변경하였다.
* 문자 앞 뒤에 공백문자가 들어가 있는 것을 strip()으로 제거한다.
* 그래도 두 번째 문자가 구가 아닌 문자들이 있다. 해당 문자들만 모아서 '구' 컬럼에 행정구가 들어갈 수 있도록 전처리 작업을 해준다.

In [ ]:
df['주소'] = df['주소'].apply(lambda x : re.sub('  ', ' ', x))

In [ ]:
df['구'] = df['주소'].apply(lambda x : x.strip().split(' ')[1])

In [ ]:
df['구'].value_counts().head()

In [ ]:
df['구'].value_counts().tail()

* 마지막 글자가 구로 끝나지 않는 데이터를 찾아온다.
* 마지막 글자를 찾을 때는 -1로 마지막 위치의 문자를 가져올 수도 있지만 
* endswith를 사용해 가져오도록 한다.

In [ ]:
# str 메소드를 사용해서 마지막 인덱스로 끝나는 글자를 가져올 수도 있지만 endswith 를 사용할 수도 있다.
# df_gu = df[df['구'].str[-1] != '구']
df_gu = df[~df['구'].str.endswith('구')]
print(df_gu.shape)
df_gu2 = df_gu['구'].unique()
df_gu2

In [ ]:
df_not_gu = df_gu[df_gu['구'].isin(df_gu2)]
df_not_gu.head()

In [ ]:
# 주소 정보가 잘못된 데이터를 찾는다.
df_not_gu['구'].unique()

* 마지막 글자가 구로 끝나지 않는 주소를 가져와서 어떤 구에 속하는지를 찾는다.

In [ ]:
dong = {'성내동':'강동구', '상일동':'강동구'}
dong['성내동']

In [ ]:
# 구 컬럼에 추출된 정보가 어떤 구에 해당되는지 전처리 해준다.
gu = {'성내동':'강동구', '상일동':'강동구', '명일동':'강동구', '길동':'강동구', 
      '암사동':'강동구', '고덕동':'강동구', '강일동':'강동구', '양재대로':'강동구', 
      '서울':'송파구', '중화동':'중랑구', '면목동':'중랑구', '상봉동':'중랑구', 
      '미아동':'강북구', '수유동':'강북구', '번동':'강북구', '동일로':'노원구', 
      '창동':'도봉구', '망원2빗물펌프장':'마포구', '가산동':'금천구', 
      '시흥동':'금천구', '독산동':'금천구', '사당동':'동작구', '관악로':'관악구', 
      '서초동':'서초구', '잠원동':'서초구', '신원동':'서초구', '양재동':'서초구',
      '염곡동':'서초구', '상암동':'마포구', '진관동':'은평구', '응암동':'은평구', 
      '녹번동':'은평구', '불광로':'은평구', '불광동':'은평구', '증산동':'은평구', 
      '역촌동':'은평구', '갈현동':'은평구'}

In [2]:
# x[-1]
"마포구"[-1]

'구'

In [3]:
"마포구"[-1] != '구'

False

In [ ]:
df['구'] = df['구'].apply(lambda x : gu[x] if x[-1] != '구' else x )

In [ ]:
# Pandas 로 시각화를 위해 데이터를 집계한다.
df_gu = df['구'].value_counts()
print("데이터에 있는 구의 수 : ", len(df_gu))
df_gu

* '구' 컬럼을 전처리한 데이터를 저장한다.

In [ ]:
df.to_csv('data/bike_rent_station_gu.csv', index=False)

In [ ]:
# Pandas로 시각화 하기
df_gu.plot(kind='bar', rot=30, figsize=(15, 5), title="구별 대여소 수")

In [ ]:
# 위와 같은 시각화를 seaborn 으로 보기
plt.figure(figsize=(15, 5))
plt.xticks(rotation=30)
sns.countplot(data=df, x='구')

In [ ]:
# 잘못입력된 위경도로 인해 그래프가 제대로 그려지지 않는 문제를 위해 위경도 범위를 지정
geo_gu_df = df.loc[(df['위도'] > 37) & (df['경도'] > 125)]
geo_gu_df.shape

In [ ]:
# 위경도에 오류가 있는 데이터를 제거한 geo_gu_df를 사용해 시각화 한다.
plt.figure(figsize=(15, 10))
sns.scatterplot(data=geo_gu_df, x='경도', y='위도', hue='구')

In [ ]:
# 위도 경도의 오류 데이터 제거를 위해 구간에 있는 데이터로 위경도의 평균을 구해온다.
df_gu_lat_long = geo_gu_df.groupby(["구"])["위도", "경도"].mean()
df_gu_lat_long.head()

In [ ]:
# 구별 대여소 수를 집계한다.
df_gu_count = pd.DataFrame(df["구"].value_counts())
df_gu_count.columns = ["대여소수"]
df_gu_count.head()

In [ ]:
# 구별 대여소 수를 비교해 본다.
plt.figure(figsize=(15, 10))
sns.scatterplot(data=geo_gu_df, x='경도', y='위도', hue='구')
sns.scatterplot(data=df_gu_lat_long, x="경도", y="위도",
                size=df_gu_count["대여소수"], hue=df_gu_count["대여소수"],
                sizes=(100, 500))

## folium 으로 지도에 시각화 하기
* [folium 예제 참고](http://nbviewer.jupyter.org/github/python-visualization/folium/tree/master/examples/)
    - 맵스타일, 히트맵, 마커 등을 커스텀하게 표현한 예제

In [ ]:
%time
geo_df = geo_gu_df
map = folium.Map(location=[geo_df['위도'].mean(), geo_df['경도'].mean()], zoom_start=12)

for n in geo_df.index:
    loc_name = geo_df.loc[n, '대여소']
    
    icon_color = 'green'
    folium.Circle(
        location=[geo_df.loc[n, '위도'], geo_df.loc[n, '경도']],
        popup=loc_name,
        radius=20,
        color=icon_color,
        fill=True,
        fill_color=icon_color
    ).add_to(map)
    
map

## 튜토리얼 만족도 설문

튜토리얼에 참여해 주셔서 감사합니다. 설문을 부탁드립니다. https://forms.gle/LB4dAkaNhwFHdbTD7